### Modified Berggren Frost Depth
#### Background

From Bianchini and Gonzalez (2012)<sup>[[6]](#6)</sup>:

> "The design of pavement structures in cold climates must account for the changes in soil properties due to the influence of freezing and thawing cycles. The calculation of frost depth is a fundamental step during the design and evaluation of pavement structures by the U.S. Department of Defense (DoD). The DoD uses the modified Berggren (ModBerg) equation to compute the frost penetration depth."
> 
The functions provided here compute the ModBerg frost depth in the same fashion as demonstrated in the The Unified Facilities Criteria (UFC) 3-130-06<sup>[[5]](#5)</sup>. Note that other tools (in particular the Pavement-Transportation Computer Assisted Structural Engineering (PCASE)) offer numerical solutions to the ModBerg equation and thus predict different values than the solutions described in the UFC and presented here. See Bianchini and Gonzalez (2012)<sup>[[6]](#6)</sup> for a discussion of those differences.

#### Assumptions

- Thaw depth is not yet implemented.
- Frost depth is computed for a single layer of homogenous isotropic soil.
- Heat flow is one-dimensional with the entire soil mass at its mean annual temperature prior to the start of the freezing season. The surface temperature changes suddenly as a step function from the mean annual temperature to a temperature v<sub>s</sub> degrees below freezing and remains at this new temperature throughout the entire freezing season.
- The coefficient λ considers the effect of the temperature changes in the soil mass and is a function of the fusion parameter μ and thermal ratio α. Traditionally, λ is found "manually" by using values of μ and α to consult a chart such as this one from Aldrich and Paynter (1966)<sup>[[3]](#3)</sup>:

 <img src="static/correction_coeff.png" width="400">

Here λ is found using equation _2-33_ from Aldrich (1953)<sup>[[2]](#2)</sup> implemented like so:

```python
def compute_lambda_coeff(mu, thermal_ratio):
    """Compute the lambda coefficient (dimensionless).
    Args:
        mu: the fusion parameter (dimensionless)
        thermal_ratio: thermal ratio (dimensionless)
    Returns
        lc: the lambda coefficient value (dimensionless).
    """
    lc = 1.0 / (np.sqrt(1 + (mu * (thermal_ratio + 0.5))))
    return round(lc, 2)
```

This method removes manual consultation of the above figure and should produce coefficients suitable for high latitudes (where the thermal ratio is low), though may overestimate frost depths for more temperate climates.

#### References

<a id="1">[1]</a> W. P. Berggren, “Prediction of temperature-distribution in frozen soils,” Trans. AGU, vol. 24, no. 3, p. 71, 1943, doi: 10.1029/TR024i003p00071.

<a id="2">[2]</a> H. P. Aldrich and H. M. Paynter, “Analytical Studies of Freezing and Thawing of Soils,” Arctic Construction and Frost Effects Laboratory, Corps of Engineers, U.S. Army, Boston, MA, First Interim Technical Report 42, Jun. 1953.

<a id="3">[3]</a> H. P. Aldrich and H. M. Paynter, “Depth of Frost Penetration in Non-Uniform Soil,” Conducted for CORPS OF ENGINEERS, U. S. ARMY by U.S. ARMY MATERIEL COMMAND COLD REGIONS RESEARCH & ENGINEERING LABORATORY, Hanover, New Hampshire, Special Report 104, Oct. 1966.

<a id="4">[4]</a> “ARCTIC AND SUBARCTIC CONSTRUCTION CALCULATION METHODS FOR DETERMINATION OF DEPTHS OF FREEZE AND THAW IN SOILS,” Joint Departments of the Army and Air Force, USA, Technical Manual Tm 5-852-6 / AFR 88-19, Volume 6, Jan. 1988.

<a id="5">[5]</a> “Calculation Methods for Determination of Depth of Freeze and Thaw in Soil: Arctic and Subarctic Construction,” U.S. Army Corps of Engineers, United Facilities Criteria UFC 3-130-06, Jan. 2004. [Online]. Available: https://www.wbdg.org/FFC/DOD/UFC/INACTIVE/ufc_3_130_06_2004.pdf

<a id="6">[6]</a> A. Bianchini and C. R. Gonzalez, “Pavement-Transportation Computer Assisted Structural Engineering (PCASE) Implementation of the Modified Berggren (ModBerg) Equation for Computing the Frost Penetration Depth within Pavement Structures,” Geotechnical and Structures Laboratory U.S. Army Engineer Research and Development Center, Vicksburg, MS, Final ERDC/GSL TR-12-15, Apr. 2012. doi: 10.21236/ADA559915.

<a id="7">[7]</a> H. P. Aldrich, “Frost Penetration Below Highway And Airfield Pavements,” p. 26.

### Usage & Introduction to Jupyter Notebooks

[Jupyter Notebooks](https://jupyter.org/) are a popular tool for interactive computing used by data professionals. Notebooks provide a versatile environment for writing and executing code, creating visualizations, and documenting your work.

#### Running Individual Cells

Jupyter Notebooks are organized into cells. Each cell can contain code or text. To run an individual code cell:
1. Click on the cell you want to run. The cell will be highlighted.
2. Press `Shift + Enter` on your keyboard, or click the "Run" button in the toolbar at the top. The code in the cell will execute, and any output (such as printed text or graphs) will be displayed immediately below the cell.
3. You can run cells in any order, making it easy to experiment and iterate with your code. Note that the notebook here expects cells to be executed in order to produce the intended results.

To run all cells in the notebook in sequence:
1. Go to the "Cell" menu at the top.
2. Select "Run All." This will execute all the cells in the notebook from top to bottom.
3. Running all cells is useful when you want to ensure that your notebook runs correctly from start to finish.

#### Downloading a Jupyter Notebook

When you want to save a local copy of the notebook:
1. Go to the "File" menu.
2. Select "Download" - this will save your notebook in the native Jupyter Notebook format (**.ipynb**). and the notebook will be downloaded to your computer.

#### Inputs
- Soil Factors
  - Average Thermal Conductivity of Frozen and Thawed Soil (BTU/hr • ft • °F)
  - Dry Unit Weight (lbs per cubic foot)
  - Gravimetric Water Content (%)
- Climate Factors Independent of SNAP Data
  - Duration of the Freezing Season (days)
  - N-factor to Convert Air Temperature and Air Freezing Index to Soil Surface Temperature and Surface Freezing Index 
- Climate Factors from the SNAP Data API
  - Emissions Scenario for Projected Mean Annual Temperature
  - Start and End Years (period over which to average mean annual temperature and freezing index projections)

In [ ]:
"""
Note: Imperial units are used to align with American engineering conventions.
"""
import micropip
await micropip.install("pyodide-http")
await micropip.install("ipywidgets")

import pyodide_http
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, Output

pyodide_http.patch_all()

scenarios = ["rcp45", "rcp85"]

def compute_volumetric_latent_heat_of_fusion(dry_ro, wc_pct):
    """Compute amount of heat required to melt all ice or freeze pore water in a unit volume of soil.

    Args:
        dry_ro: soil dry unit weight (lbs per cubic foot)
        wc_pct: water content (percent)
    Returns:
        L: volumetric latent heat of fusion (BTUs per cubic foot)
    """
    # latent heat of fusion for ice is 144 BTU/lb
    # 144 BTU are absorbed for each pound of ice that is melted to water.
    L = 144 * dry_ro * (wc_pct / 100)
    L = int(round(L))
    print(f"Computed Volumetric Latent Heat of Fusion is {L} BTUs per cubic foot")
    return L


def compute_frozen_volumetric_specific_heat(dry_ro, wc_pct):
    """Compute quantity of heat required to change temperature of a frozen unit volume of soil by 1°F. 

    Args:
        dry_ro: soil dry unit weight (lbs per cubic foot)
        wc_pct: gravimetric water content (percent)
    Returns:
        c: volumetric specific heat ((BTUs per cubic foot) • °F)
    """
    # specific heat of soil solids is 0.17 (BTU/lb • °F) for most soils
    c = dry_ro * (0.17 + (0.5 * (wc_pct / 100)))
    c = round(c, 1)
    print(f"Computed Frozen Soil Volumetric Specific Heat is {c} (BTUs per cubic foot) • °F")
    return c


def compute_unfrozen_volumetric_specific_heat(dry_ro, wc_pct):
    """Compute quantity of heat required to change the temperature of an unfrozen unit volume of soil by 1°F. The specific heat of soil solids is 0.17 BTU/lb • °F for most soils.

    Args:
        dry_ro: soil dry unit weight (lbs per cubic foot)
        wc_pct: gravimetric water content (percent)
    Returns:
        c: volumetric specific heat (BTUs per cubic foot) • °F
    """
    # specific heat of soil solids is 0.17 (BTU/lb • °F) for most soils
    c = dry_ro * (0.17 + (1.0 * (wc_pct / 100)))
    c = round(c, 1)
    print(f"Computed Thawed Soil Volumetric Specific Heat is {c} (BTUs per cubic foot) • °F")
    return c


def compute_avg_volumetric_specific_heat(dry_ro, wc_pct):
    """Compute quantity of heat required to change the temperature of an average unit volume of soil by 1°F. The specific heat of soil solids is 0.17 BTU/lb • °F for most soils.

    Args:
        dry_ro: soil dry unit weight (lbs per cubic foot)
        wc_pct: gravimetric water content (percent)
    Returns:
        c: volumetric specific heat (BTUs per cubic foot) • °F
    """
    # specific heat of soil solids is 0.17 (BTU/lb • °F) for most soils
    c = dry_ro * (0.17 + (0.75 * (wc_pct / 100)))
    c = round(c, 1)
    print(f"Computed Soil (Average of Frozen and Thawed) Volumetric Specific Heat is {c} (BTUs per cubic foot) • °F")
    return c


def compute_seasonal_v_s(nFI, d):
    """Compute the v_s parameter. v_s has two possible meanings depending on the problem being studied. In this case, v_s is used for computing a seasonal depth of freeze.

    Args:
        nFI: surface freezing index (°F • days)
        d: length of freezing duration (days)
    Returns:
        v_s: parameter describing intensity of freezing season
    """
    v_s = nFI / d
    return v_s


def get_projected_maat_from_api(lat, lon, scenario, year_start, year_end):
    """Query the SNAP Data API for mean annual air temperature.

    Args:
        lat: valid SNAP Data API latitude
        lon: valid SNAP Data API longitude
        scenario (str): one of [rcp45, rcp85]
        year_start (int): start year for summary period
        year_end (int): end year for summary period
    Returns:
        maat_deg_F: mean annual temperature (F) for the given scenario, averaged over the summary period and all models
    """
    # make the request
    api_url = f"https://earthmaps.io/temperature/{lat}/{lon}/{year_start}/{year_end}?format=csv"
    df = pd.read_csv(api_url, header=3)
    # get the mean of the mean annual air temperature values for all years in the time span, for the specific emissions scenario
    maat_deg_C = df.groupby(["scenario"])["tas"].mean().loc[scenario]
    # convert from °C to °F
    maat_deg_F = (maat_deg_C * 1.8) + 32
    maat_deg_F = float(round(maat_deg_F, 1))
    print(f"SNAP Data API Mean Annual Air Temperature: {maat_deg_F} °F")
    return maat_deg_F


def get_projected_freezing_index_from_api(lat, lon, scenario, year_start, year_end):
    """Query the SNAP Data API for cumulative air freezing index values.

    Args:
        lat: valid SNAP Data API latitude
        lon: valid SNAP Data API longitude
        scenario (str): one of [rcp45, rcp85]
        year_start (int): start year for summary period
        year_end (int): end year for summary period
    Returns:
        freezing_index_degFdays: annual cumulative freezing index averaged over all models for the specified year range and scenario
    """
    api_url = f"https://earthmaps.io/degree_days/freezing_index/{lat}/{lon}/{year_start}/{year_end}?format=csv"
    df = pd.read_csv(api_url, header=3)
    freezing_index_degFdays = int(round(df.groupby(["scenario"])["dd"].mean().loc[scenario]))
    print(f"SNAP Data API Mean Cumulative Air Freezing Index: {freezing_index_degFdays} °F⋅days")
    return freezing_index_degFdays


def compute_multiyear_v_s(maat):
    """Compute the v_s parameter.

    v_s has one of two possible meanings depending on the problem being studied. In this case, v_s is useful in computing multiyear freeze depths that may develop as a long-term change in the heat balance at the ground surface.

    Args:
        maat: mean annual air temperature (°F)
    Returns:
         v_s
    """
    v_s = abs(maat - 32)
    return v_s


def compute_v_o(maat):
    """Compute the v_o parameter.

    v_o is the absolute value of the difference between the mean annual temperature BELOW THE GROUND SURFACE and 32 °F.

    Args:
        magt: mean annual temperature below the ground surface (°F)
    Returns:
         v_o
    """
    v_o = abs(maat - 32)
    return v_o


def compute_thermal_ratio(v_o, v_s):
    """Compute the the thermal ratio.

    The thermal ratio is the ratio of two deltas: ground temperature - freezing and surface temperature - freezing.

    Args:
        v_o
        v_s
    Returns:
        thermal_ratio: dimensionless
    """
    thermal_ratio = v_o / v_s
    thermal_ratio = round(thermal_ratio, 2)
    print(f"Computed thermal ratio: {thermal_ratio}")
    return thermal_ratio


def compute_fusion_parameter(v_s, c, L):
    """Compute the fusion parameter.

    Args:
        v_s: v_s parameter
        c: volumetric specific heat ((BTUs per cubic foot) • °F)
        L: volumetric latent heat of fusion (BTUs per cubic foot)
    Returns:
        mu: (dimensionless)
    """
    mu = v_s * (c / L)
    mu = round(mu, 2)
    print(f"Computed fusion parameter: {mu}")
    return mu


def compute_lambda_coeff(mu, thermal_ratio):
    """Compute the lambda coefficient (dimensionless).

    Reference: H. P. Aldrich and H. M. Paynter, “Analytical Studies of Freezing and Thawing of Soils,” Arctic Construction and Frost Effects Laboratory, Corps of Engineers, U.S. Army, Boston, MA, First Interim Technical Report 42, Jun. 1953.

    Other implementations:
    `lc2 = 0.707 / (np.sqrt(1 + (mu * (thermal_ratio + 0.5))))`
    `lc_mean = np.mean([lc, lc2])`

    lc (used here) may overestimate frost depth, but is suited to high latitudes
    lc2 may underestimate frost depth, but is suited to lower latitudes
    lc_mean is a middle ground

    Args:
        mu: fusion parameter (dimensionless)
        thermal_ratio: thermal ratio (dimensionless)
    Returns
        lc: the lambda coefficient (dimensionless)
    """
    lc = 1.0 / (np.sqrt(1 + (mu * (thermal_ratio + 0.5))))
    lc = round(lc, 2)
    print(f"Computed lambda coefficient: {lc}")
    return lc


def compute_depth_of_freezing(coeff, k_avg, nFI, L):
    """Compute the depth to which 32 °F temperatures will penetrate into the soil mass.

    Args:
        coeff: the lambda coefficient (dimensionless)
        k_avg: thermal conductivity of soil, average of frozen and unfrozen (BTU/hr • ft • °F)
        nFI: surface freezing index (°F • days)
        L: volumetric latent heat of fusion (BTUs per cubic foot)
    Returns:
        x: frost depth (feet)
    """
    x = coeff * np.sqrt((48 * k_avg * nFI) / L)
    return round(x, 1)


def compute_modified_berggren(
    dry_ro,
    wc_pct,
    d,
    n_index,
    n_temp,
    k_avg,
    lat,
    lon,
    scenario,
    year_start,
    year_end,
):
    """
    Args:
        dry_ro: soil dry unit weight (lbs per cubic foot)
        wc_pct: gravimetric water content (percent)
        d: length of freezing duration (days)
        n_index: factor to convert air to surface freezing index 
        n_temp: factor to convert mean annual air temperature to mean annual soil surface temperature
        k_avg: thermal conductivity of soil, avg of frozen and unfrozen (BTU/hr • ft • °F)
        lat: valid latitude for SNAP Data API
        lon: valid longitude for SNAP Data API
        scenario: emissions scenario for mean annual air temperature and air freezing index for SNAP Data API
        year_start: start year for average summary of temperature and freezing index
        year_end: end year for average summary of temperature and freezing index
    Returns:
        frost_depth: Modified Berggren frost depth (feet)
    """
    maat = get_projected_maat_from_api(lat, lon, scenario, year_start, year_end)
    masst = n_temp * maat
    masst = round(masst, 1)
    print(f"Mean Annual Soil Surface Temperature After Applying N-Factor to SNAP Data API Mean Annual Air Temperature : {masst} °F")

    freezing_index = get_projected_freezing_index_from_api(lat, lon, scenario, year_start, year_end)
    surface_freezing_index = int(round(n_index * freezing_index))
    print(f"Cumulative Surface Freezing Index After Applying N-Factor to SNAP Data API Air Freezing Index : {surface_freezing_index} °F⋅days")

    L = compute_volumetric_latent_heat_of_fusion(dry_ro, wc_pct)
    c = compute_avg_volumetric_specific_heat(dry_ro, wc_pct)
    v_s = compute_seasonal_v_s(surface_freezing_index, d)
    v_o = compute_v_o(masst)
    
    thermal_ratio = compute_thermal_ratio(v_o, v_s)
    mu = compute_fusion_parameter(v_s, c, L)
    lambda_coeff = compute_lambda_coeff(mu, thermal_ratio)

    frost_depth = compute_depth_of_freezing(lambda_coeff, k_avg, surface_freezing_index, L)
    return frost_depth

In [ ]:
# input for SNAP Data API Queries
lat_box = widgets.BoundedFloatText(
    value=66.0,
    min=51.3,
    max=71.3,
    step=0.1,
    description="Latitude",
    continuous_update=False,
)
lon_box = widgets.BoundedFloatText(
    value=-147.0,
    min=-179.1,
    max=-130.0,
    step=0.1,
    description="Longitude",
    continuous_update=False,
)
scenario_dropdown = widgets.Dropdown(
    options=scenarios,
    description="Emissions Scenario for Climate Model Projections:",
    layout=Layout(width="400px"),
    style={"description_width": "initial"},
)
start_slider = widgets.IntSlider(
    value=2030,
    min=2018,
    max=2098,
    description="Start Year:",
    layout=Layout(width="400px"),
)
stop_slider = widgets.IntSlider(
    value=2059,
    min=2019,
    max=2099,
    description="End Year:",
    layout=Layout(width="400px"),
)
# soil property inputs
dry_ro_slider = widgets.IntSlider(
    value=100,
    min=20,
    max=135,
    step=1,
    description="Soil Dry Unit Weight (lbs per cubic foot):",
    layout=Layout(width="400px"),
    style={"description_width": "initial"},
)
wc_pct_slider = widgets.IntSlider(
    value=15,
    min=1,
    max=50,
    step=1,
    description="Soil Gravimetric Water Content (%):",
    layout=Layout(width="400px"),
    style={"description_width": "initial"},
)
k_avg_slider = widgets.FloatSlider(
    value=0.78,
    min=0.1,
    max=2,
    step=0.1,
    description="Average Thermal Conductivity of Frozen and Thawed Soil (BTU/hr • ft • °F):",
    layout=Layout(width="700px"),
    style={"description_width": "initial"},
)
# other climate inputs
d_slider = widgets.IntSlider(
    value=200,
    min=1,
    max=365,
    step=1,
    description="Freezing Duration (days):",
    layout=Layout(width="400px"),
    style={"description_width": "initial"},
)
n_index_slider = widgets.FloatSlider(
    value=0.75,
    min=0.10,
    max=1.5,
    step=0.01,
    description="N-factor to Convert Air Index Values to Surface Index Values:",
    layout=Layout(width="700px"),
    style={"description_width": "initial"},
)
n_temp_slider = widgets.FloatSlider(
    value=0.85,
    min=0.10,
    max=1.5,
    step=0.01,
    description="N-factor to Convert Air Temperature to Surface Temperature:",
    layout=Layout(width="700px"),
    style={"description_width": "initial"},
)

# create output widget for displaying the result
output = Output()
# function to compute and display the result
def compute_and_display():
    with output:
        output.clear_output()
        result = compute_modified_berggren(
            dry_ro=dry_ro_slider.value,
            wc_pct=wc_pct_slider.value,
            d=d_slider.value,
            n_index=n_index_slider.value,
            n_temp=n_temp_slider.value,
            k_avg=k_avg_slider.value,
            lat=lat_box.value,
            lon=lon_box.value,
            scenario=scenario_dropdown.value,
            year_start=start_slider.value,
            year_end=stop_slider.value,
        )
        print(f"Modified Berggren Depth: {result} feet")


# create button to trigger the computation
compute_button = widgets.Button(description="Compute Modified Berggren", layout=Layout(width="400px"))
compute_button.on_click(lambda btn: compute_and_display())
# create container to organize the widgets
widget_container = widgets.VBox(
    [
        widgets.Label(value="The following location, time, and emissions scenario parameters are used to fetch air temperature and air freezing index values from downscaled climate model output via the SNAP Data API:"),
        lat_box,
        lon_box,
        start_slider,
        stop_slider,
        scenario_dropdown,
        widgets.Label(value="Specify the following required climate and environmental parameters that are not supplied by the SNAP Data API:"),
        d_slider,
        n_index_slider,
        n_temp_slider,
        widgets.Label(value="Specify the following soil parameters:"),
        dry_ro_slider,
        wc_pct_slider,
        k_avg_slider,
        compute_button,
        output,
    ]
)
widget_container